In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


pdf = "Error_PDFS/AZD_09_17_2022.pdf"
#pdf = "Example_PDFS/equibaseFile.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 6

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan

The PDF <_io.BufferedReader name='Error_PDFS/AZD_09_17_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


['ARIZONA', 'DOWN', '/S', '-', 'September', '17,', '2022', '-', 'Race', '7', 'MAIDEN', 'OPTIONAL', 'CLAIMING', '-', 'Thoroughbred', 'FOR', 'MAIDENS,', 'FILLIES', 'AND', 'MARES', 'THREE', 'YEARS', 'OLD', 'AND', 'UPWARD.', 'Three', 'Year', 'Olds,', '120', 'lbs.;', 'Older,', '124', 'lbs.', 'Claiming', 'Price', '$30,000.', 'Claiming', 'Price', '$30,000', 'Five', 'And', 'One', 'Half', 'Furlongs', 'On', 'The', 'Dirt', 'Current', 'Track', 'Record', '(', 'Botero', '-', '1', '01.56', '-', 'August', '17,', '2021)', 'Purse', '$14,025', 'Includes', '$5,175', 'Other', 'Sources', 'Includes', '$5,400', 'Other', 'Sources', 'Available', 'Money', '$14,025', 'Value', 'of', 'Race', '$14,025', '1st', '$8,415,', '2nd', '$2,805,', '3rd', '$1,403,', '4th', '$561,', '5th', '$421,', '6th', '$280,', '7th', '$140', 'Weather', 'Cloudy', 'Track', 'Fast', 'Off', 'at', '4', '51', 'Start', 'Good', 'for', 'all', 'except', '1', 'Timer', 'Hand']
track :  ARIZONA DOWN /S
date :  September 17, 2022
race_num :  7
criteria :

In [4]:
result_tables[1]

,Pgm,Horse Name,Start,1/4,3/8,Str,Fin,1/4_length_behind,3/8_length_behind,Str_length_behind,Fin_length_behind
0,4,Boisterously Irish,4.0,5,3,2,1,5,3 1/2,2,1 3/4
1,3,She Had a Secret,3.0,2,1,1,2,1 1/2,Head,2,1 3/4
2,2,Belle My Belle,5.0,4,4,3,3,4,3 3/4,2 1/2,3
3,6,Downtown Cutie,2.0,1,2,4,4,1 1/2,Head,5 1/2,11 1/2
4,1,Sweet and Ez,7.0,6,5,5,5,6,5 3/4,10,14 1/4
5,5,C'Mon Lets Switch,1.0,3,6,6,6,3 1/2,7 1/4,17,31
6,7,Legal Mischief,6.0,---,---,---,---,NaN,NaN,NaN,NaN


In [ ]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    print(header)
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0].astype(object)
    bottom_table = result_tables[1].astype(object)
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/AZD_09_17_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 8}, {'page_num': 1, 'horse_count': 9}, {'page_num': 2, 'horse_count': 9}, {'page_num': 3, 'horse_count': 7}, {'page_num': 4, 'horse_count': 7}, {'page_num': 5, 'horse_count': 7}, {'page_num': 6, 'horse_count': 7}, {'page_num': 7, 'horse_count': 8}, {'page_num': 8, 'horse_count': 8}]
['ARIZONA', 'DOWNS', '[-', 'September', '17,', '2022', '-', 'Race', '1', 'ALLOWANCE', '-', 'Quarter', 'Horse', 'FOR', 'THREE', 'YEAR', 'OLDS', 'AND', 'UPWARD', 'WHICH', 'HAVE', 'NEVER', 'WON', 'TWO', 'RACES.', 'Three', 'Year', 'Olds,', '124', 'lbs.;', 'Older,', '126', 'lbs.', '(NW2', 'L)', 'Four', 'Hundred', 'Yards', 'On', 'The', 'Dirt', 'Current', 'Track', 'Record', '(First', 'Lady', 'Perry', '-', '19.233', '-', 'September', '1,', '2019)', 'Purse', '$8,100', 'Includes', '$4,600', 'Other', 'Sources', 'Available', 'Money', '$8,100', 'Value', 'of', 'Race', '$8,100', '1st', '$4,455,', '2nd', '$1,620,', '3rd', '$810,', '4th', '$405,', '5th', '$324,', '6th', '$162,', '7th', '$162,

The PDF <_io.BufferedReader name='Error_PDFS/AZD_09_17_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 463.29200000000003
    Last Raced  Pgm jockey_first_name jockey_last_name   PP Start Str 1 Str  \
0   3Sep22 AZD  6.0           Rodolfo           Arvizu  6.0   4.0     1   1   
1   3Sep22 AZD  7.0             Elias        Gutierrez  7.0   2.0     4   2   
2   3Sep22 AZD  3.0             David     Reyes-Frisby  3.0   5.0     3   3   
3  15Aug22 AZD  1.0           Michael           Ybarra  1.0   1.0     2   4   
4  15Aug22 AZD  8.0            Martin            Osuna  8.0   3.0     5   5   
5  27Aug22 AZD  4.0              Jose             Leon  4.0   7.0     6   6   
6  15Aug22 AZD  5.0             Aaron           Belloc  5.0   8.0     7   7   
7  23Jul22 AZD  2.0             Kevin         Carbajal  2.0   6.0     8   8   

  Fin   Odds  ...                date race_num  \
0   1   3.90  ...  September 17, 2022        1   
1   2  15.30  ...  September 17, 2022        1   
2   3  1.70*  ...  September 17, 2022        1   
3   4   7.80  ...  September 17, 2022    

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/AZD_09_17_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/AZD_09_17_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


['ARI', '[ZONA', 'DOWNS', '-', 'September', '17,', '2022', '-', 'Race', '6', 'CLAIMING', '-', 'Thoroughbred', 'FOR', 'THREE', 'YEAR', 'OLDS', 'AND', 'UPWARD.', 'Three', 'Year', 'Olds,', '120', 'lbs.;', 'Older,', '124', 'lbs.', 'Claiming', 'Price', '$15,000.', 'Arizona', 'Bred', 'Claiming', 'Price', '$20,000.', 'Claiming', 'Price', '$20,000', '-', '$15,000', 'Two', 'Furlongs', 'On', 'The', 'Dirt', 'Current', 'Track', 'Record', '(', 'Master', 'Ruler', '-', '22.34', '-', 'September', '17,', '2022)', 'Purse', '$15,025', 'Includes', '$5,775', 'Other', 'Sources', 'Includes', '$5,400', 'Other', 'Sources', 'Available', 'Money', '$15,025', 'Value', 'of', 'Race', '$15,025', '1st', '$9,015,', '2nd', '$3,005,', '3rd', '$1,503,', '4th', '$601,', '5th', '$451,', '6th', '$300,', '7th', '$150', 'Weather', 'Cloudy', 'Track', 'Fast', 'Off', 'at', '4', '16', 'Start', 'Good', 'for', 'all', 'Timer', 'Hand']
track :  ARI [ZONA DOWNS
date :  September 17, 2022
race_num :  6
criteria :  FOR THREE YEAR OLDS AN

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/AZD_09_17_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 504.858
     Last Raced   Pgm jockey_first_name jockey_last_name   PP Start Str 1 Str  \
0    3Sep22 AZD   6.0           Rodolfo           Arvizu  6.0   4.0     1   1   
1    3Sep22 AZD   7.0             Elias        Gutierrez  7.0   2.0     4   2   
2    3Sep22 AZD   3.0             David     Reyes-Frisby  3.0   5.0     3   3   
3   15Aug22 AZD   1.0           Michael           Ybarra  1.0   1.0     2   4   
4   15Aug22 AZD   8.0            Martin            Osuna  8.0   3.0     5   5   
5   27Aug22 AZD   4.0              Jose             Leon  4.0   7.0     6   6   
6   15Aug22 AZD   5.0             Aaron           Belloc  5.0   8.0     7   7   
7   23Jul22 AZD   2.0             Kevin         Carbajal  2.0   6.0     8   8   
8   15Aug22 AZD   3.0             Kevin         Carbajal  3.0   1.0   NaN   1   
9   27Aug22 AZD   1.0              Kody     Kellenberger  1.0   5.0   NaN   2   
10  29Aug22 AZD   9.0           Rodolfo           Arvizu  8.0   3.0   NaN

Looking for tables...
Found table top bound at 143.82999999999993
Found table bottom bound at 235.13
Looking for tables...


In [6]:
pdf_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,Start,1/4,1/2,Str,Fin,...,Str 1_length_behind_RLP,claiming_price,3/16,3/8,3/16_length_behind,3/8_length_behind,3/16_RLP,3/8_RLP,3/16_length_behind_RLP,3/8_length_behind_RLP
0,15Aug22 ARP,5.0,Hugo,Herrera,5.0,2.0,2,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11Sep22 ARP,2.0,David,Martin,2.0,5.0,5,6,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25Sep22 ARP,4.0,Nathan,Haar,4.0,4.0,6,5,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12Sep22 ARP,3.0,Adrian,Ramos,3.0,1.0,1,1,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8Aug22 ARP,6.0,Travis,Wales,6.0,6.0,4,3,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7Aug22 ARP,1.0,Pedro,Gonzalez,1.0,3.0,3,4,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,18Sep22 ARP,7.0,Adrian,Morales,7.0,1.0,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4Sep22 ARP,2.0,Martin,Lazalde,2.0,3.0,NaN,NaN,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,18Sep22 ARP,1.0,Jose,Lopez,1.0,2.0,NaN,NaN,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,28Aug22 ARP,4.0,Jhaciel,Soto,4.0,4.0,NaN,NaN,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pdf_df.to_csv("ARAP.csv")